<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Scrapping Data From a Wiki Page</a>

2. <a href="#item2">Exploring Neighborhoods</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Clustering Neighborhoods</a>

5. <a href="#item5">Examining Clusters</a>    
</font>
</div>

## 1. Scrapping Data From a Wiki Page

Scrapping table info from a wiki page

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

text = soup.find('table').tbody.text.strip()
text = text.split('\n\n\n')
text[:5]

['Postcode\nBorough\nNeighbourhood',
 'M1A\nNot assigned\nNot assigned',
 'M2A\nNot assigned\nNot assigned',
 'M3A\nNorth York\nParkwoods',
 'M4A\nNorth York\nVictoria Village']

Creating a pandas DataFrame object

In [2]:
postcodes = []
boroughs = []
neighborhoods = []

for line in text:
    line = line.split('\n')
    postcodes.append(line[0])
    boroughs.append(line[1])
    neighborhoods.append(line[2])
    

data = pd.DataFrame({'PostalCode':postcodes, 'Borough':boroughs, 'Neighborhood':neighborhoods})
data.drop([0], inplace=True)
print('Initial shape of the table: ', data.shape)
data.head(7)

Initial shape of the table:  (287, 3)


,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


Cleaning table

In [3]:

#droping rows with Borough == 'Not assigned'
print('Num of rows with Borough name \'Not assigned\': ', sum(data['Borough']=='Not assigned'))
data.drop(data[data['Borough']=='Not assigned'].index , inplace=True)

#in rows where Neighborhood == 'Not assigned'
#change the Neighborhood name to Boruogh name
print('Num of rows with Neighborhood name \'Not assigned\': ', data[data['Neighborhood'] == 'Not assigned'].shape[0])


Num of rows with Borough name 'Not assigned':  77
Num of rows with Neighborhood name 'Not assigned':  0


Joining rows with same PostalCode

In [4]:
postalcodes = data['PostalCode'].unique()
print('num of unique PostalCodes: ', len(postalcodes))

postcodes = []
boroughs = []
neighborhoods = []

for postalcode in postalcodes:
    
    postcodes.append(postalcode)
    boroughs.append(data[data['PostalCode'] == postalcode]['Borough'].values[0])
    neighborhood = data[data['PostalCode'] == postalcode]['Neighborhood'].values
    neighborhoods.append(', '.join(neighborhood))

data = pd.DataFrame({'PostalCode':postcodes, 'Borough':boroughs, 'Neighborhood':neighborhoods})
data.head(7)

num of unique PostalCodes:  103


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"


In [5]:
print('final shape: ',data.shape)

final shape:  (103, 3)


#### Getting geo coordinates

In [6]:
coords = pd.read_csv('http://cocl.us/Geospatial_data')
coords.shape

(103, 3)

In [7]:
data = data.merge(coords, left_on='PostalCode', right_on='Postal Code')
data.drop(columns='Postal Code', inplace=True)
data.head(7)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353


Get geo coordinates of Toronto city with geopy library

In [8]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)

latitude_tor = location.latitude
longitude_tor = location.longitude
print('The geograpical coordinates of Toronto City are {}, {}.'.format(latitude_tor, longitude_tor))

The geograpical coordinates of Toronto City are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [9]:
import folium
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_tor, longitude_tor], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{} ({})'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Downtown Toronto, East and West Toronto. So let's slice the original dataframe and create a new dataframe

In [10]:
id_1 = (data['Borough']=='Downtown Toronto').values
#id_2 = (data['Borough']=='East Toronto').values
#id_3 = (data['Borough']=='West Toronto').values

new_data = data[ id_1].reset_index(drop=True)

new_data.shape

(19, 5)

In [11]:
address = 'Downtown Toronto, Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
lat_new = location.latitude
lon_new = location.longitude
print('The geograpical coordinate of Downtown, East/West Toronto are {}, {}.'.format(lat_new, lon_new))

The geograpical coordinate of Downtown, East/West Toronto are 43.6563221, -79.3809161.


In [12]:
# create map of Downtown Toronto using latitude and longitude values
map_new = folium.Map(location=[lat_new, lon_new], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(new_data['Latitude'], new_data['Longitude'], new_data['Borough'], new_data['Neighborhood']):
    label = '{} ({})'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_new)  
    
map_new

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Defining Foursquare Credentials and Version

In [ ]:
#Hide this ! or rename it
CLIENT_ID = 'your-client-id' # your Foursquare ID
CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Let's explore the first neighborhood in our dataframe.

In [14]:
new_data.loc[0,'Neighborhood']

'Harbourfront'

Get the neighborhood's latitude and longitude values.

In [15]:
neighborhood_latitude = new_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = new_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = new_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in Harbourfront within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [16]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&radius=500&limit=100&v={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION)

results = requests.get(url).json()

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [17]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


We will use `get_category_type()` function to extract only gategory name

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


And how many venues were returned by Foursquare?

In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


## 2. Exploring Neighborhoods

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT =200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #print(url)
            
        # make the GET request
        while True:
            try:
                results = requests.get(url).json()["response"]['groups'][0]['items']
                break
            except:
                print('error')
                pass
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Creating a dataframe of near by venues for ech Neighborhood

In [22]:
new_venues = getNearbyVenues(names=new_data['Neighborhood'].values,
                                   latitudes=new_data['Latitude'].values,
                                   longitudes=new_data['Longitude'].values
                                  )

Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley


In [23]:
print(new_venues.shape)
new_venues.head()

(1307, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Let's check how many venues were returned for each neighborhood

In [24]:
new_venues.groupby('Neighborhood')[['Venue']].count()

,Venue
Neighborhood,
"Adelaide, King, Richmond",100
Berczy Park,55
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14
"Cabbagetown, St. James Town",48
Central Bay Street,82
"Chinatown, Grange Park, Kensington Market",85
Christie,18
Church and Wellesley,84
"Commerce Court, Victoria Hotel",100


#### Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(new_venues['Venue Category'].unique())))

There are 206 uniques categories.


## 3. Analyze Each Neighborhood

In [26]:
# one hot encoding
new_onehot = pd.get_dummies(new_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
new_onehot['Neighborhood'] = new_venues[['Neighborhood']]

ind = (new_onehot.columns == 'Neighborhood').tolist().index(True)

# move neighborhood column to the first column
fixed_columns =  [new_onehot.columns[ind]] + list(new_onehot.columns[:ind]) + list(new_onehot.columns[ind+1:])
new_onehot = new_onehot[fixed_columns]

new_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [27]:
new_onehot.shape

(1307, 206)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
new_grouped = new_onehot.groupby('Neighborhood').mean().reset_index()
new_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,...,0.0,0.0,0.0,0.020000,0.0,0.0,0.010000,0.0,0.01,0.000000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.018182,0.0,0.0,0.000000,0.0,0.00,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.071429,0.071429,0.071429,0.142857,0.071429,0.071429,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,0.000000
3,"Cabbagetown, St. James Town",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,0.000000
4,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,0.0,...,0.0,0.0,0.0,0.012195,0.0,0.0,0.012195,0.0,0.00,0.012195


#### Let's confirm the new size

In [29]:
new_grouped.shape

(19, 206)

#### Let's print each neighborhood along with the top 5 most common venues

In [30]:
num_top_venues = 3

for hood in new_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = new_grouped[new_grouped['Neighborhood'] == hood].T.reset_index()
    #print(temp)
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1  Thai Restaurant  0.04
2       Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.04
2  Farmers Market  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
            venue  freq
0  Airport Lounge  0.14
1     Coffee Shop  0.07
2        Boutique  0.07


----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop  0.06
1   Restaurant  0.06
2         Park  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Restaurant  0.05
2                Café  0.05


----Chinatown, Grange Park, Kensington Market----
    venue  freq
0     Bar  0.07
1    Café  0.06
2  Bakery  0.05


----Christie----
           venue  freq
0  Grocery Store  0.22
1           Café  0.17
2           Park  0.11


----Church and Wellesley----
               

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [31]:
import numpy as np 

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = new_grouped['Neighborhood']

for ind in np.arange(new_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(new_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Restaurant,Bar,Thai Restaurant,Sushi Restaurant,Cosmetics Shop,Steakhouse,Burger Joint,Clothing Store
1,Berczy Park,Coffee Shop,Farmers Market,Seafood Restaurant,Bakery,Cocktail Bar,Beer Bar,Cheese Shop,Café,Restaurant,Hotel
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport,Airport Food Court
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Pub,Café,Italian Restaurant,Bakery,Pizza Place,Gym / Fitness Center,Breakfast Spot
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Burger Joint,Juice Bar,Salad Place,Bar,Bubble Tea Shop


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 4 clusters.

In [33]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 4

new_grouped_clustering = new_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(new_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 1, 1, 1, 3, 1, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

new_merged = new_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
new_merged = new_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

new_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Pub,Park,Bakery,Mexican Restaurant,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Park,Gym,Burger Joint,Seafood Restaurant,Sandwich Place,Restaurant,Burrito Place,Portuguese Restaurant,Café
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Japanese Restaurant,Cosmetics Shop,Pizza Place,Restaurant,Bookstore,Bakery
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Restaurant,Clothing Store,Bakery,Hotel,Cosmetics Shop,Breakfast Spot,Beer Bar,American Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Farmers Market,Seafood Restaurant,Bakery,Cocktail Bar,Beer Bar,Cheese Shop,Café,Restaurant,Hotel


Finally, let's visualize the resulting clusters

In [35]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[lat_new, lon_new], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))

rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(new_merged['Latitude'], new_merged['Longitude'], new_merged['Neighborhood'], new_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color='#000000',#rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
    
       
map_clusters

## 5. Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

#### Cluster 1

In [36]:
new_merged.loc[new_merged['Cluster Labels'] == 0, new_merged.columns[[1, 2] + list(range(5, new_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,Rosedale,0,Park,Playground,Trail,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


The most common venues in Rosedale are parks, playgrounds, and trails. We can also see in the map above, that the whole neighborhood is surrounded by a lot of parks. No wonder Rosedale is clustered separately from other neighborhoods because of the number of green spaces around it.

#### Cluster 2

In [37]:
new_merged.loc[new_merged['Cluster Labels'] == 1, new_merged.columns[[1, 2] + list(range(5, new_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,1,Coffee Shop,Pub,Park,Bakery,Mexican Restaurant,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market
1,Downtown Toronto,Queen's Park,1,Coffee Shop,Park,Gym,Burger Joint,Seafood Restaurant,Sandwich Place,Restaurant,Burrito Place,Portuguese Restaurant,Café
2,Downtown Toronto,"Ryerson, Garden District",1,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Japanese Restaurant,Cosmetics Shop,Pizza Place,Restaurant,Bookstore,Bakery
3,Downtown Toronto,St. James Town,1,Café,Coffee Shop,Restaurant,Clothing Store,Bakery,Hotel,Cosmetics Shop,Breakfast Spot,Beer Bar,American Restaurant
4,Downtown Toronto,Berczy Park,1,Coffee Shop,Farmers Market,Seafood Restaurant,Bakery,Cocktail Bar,Beer Bar,Cheese Shop,Café,Restaurant,Hotel
5,Downtown Toronto,Central Bay Street,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Burger Joint,Juice Bar,Salad Place,Bar,Bubble Tea Shop
7,Downtown Toronto,"Adelaide, King, Richmond",1,Coffee Shop,Café,Restaurant,Bar,Thai Restaurant,Sushi Restaurant,Cosmetics Shop,Steakhouse,Burger Joint,Clothing Store
8,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",1,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Fried Chicken Joint,Scenic Lookout,Brewery,Italian Restaurant,Park
9,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",1,Coffee Shop,Café,Hotel,Restaurant,Bakery,Italian Restaurant,Japanese Restaurant,Gastropub,Bar,Seafood Restaurant
10,Downtown Toronto,"Commerce Court, Victoria Hotel",1,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Gastropub,Japanese Restaurant,Italian Restaurant,Seafood Restaurant


The second cluster stands out from others primarily because of the number of coffee shops and restaurants it encompasses in comparison to neighborhoods in other clusters. 

#### Cluster 3

In [38]:
new_merged.loc[new_merged['Cluster Labels'] == 2, new_merged.columns[[1, 2] + list(range(5, new_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",2,Airport Lounge,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Airport,Airport Food Court


Neighborhoods in the third cluster are unique because of the number of venues related to air transportation. 

#### cluster 4

In [39]:
new_merged.loc[new_merged['Cluster Labels'] == 3, new_merged.columns[[1, 2] + list(range(5, new_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,Christie,3,Grocery Store,Café,Park,Gas Station,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Candy Store


Grocery stores and a gas station can be only found in the top 10 list of the Christie neighborhood.